In [0]:
! pip install syft

     |████████████████████████████████| 194kB 41.0MB/s 
     |████████████████████████████████| 71kB 22.4MB/s 
     |████████████████████████████████| 389kB 49.9MB/s 
     |████████████████████████████████| 204kB 48.8MB/s 
     |████████████████████████████████| 1.4MB 49.7MB/s 
     |████████████████████████████████| 450kB 49.5MB/s 
     |████████████████████████████████| 276kB 47.4MB/s 
     |████████████████████████████████| 51kB 23.4MB/s 
     |████████████████████████████████| 122kB 49.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [0]:
import torch as th
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms

import syft as sy

W0623 15:03:34.169152 140509432104832 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0-rc1.so'
W0623 15:03:34.185925 140509432104832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

# Trusted Aggregator

In [0]:

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
bob.add_workers([alice,secure_worker])
alice.add_workers([bob,secure_worker])
secure_worker.add_workers([alice,bob])

In [0]:
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]],requires_grad = True)
target = th.tensor([[0],[0],[1],[1.]],requires_grad = True)

In [0]:
bob_data = data[:2].send(bob)
bob_target = target[:2].send(bob)

In [0]:
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [0]:
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [0]:
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=.1)

In [0]:
# for i in range(10):
    
#     bobs_opt.zero_grad()
#     bobs_pred = bobs_model(bob_data)
#     bobs_loss = ((bobs_pred - bob_target)**2).sum()
#     bobs_loss.backward()
#     bobs_opt.step()

#     alices_opt.zero_grad()
#     alices_pred = alices_model(alice_data)
#     alices_loss = ((alices_pred - alice_target)**2).sum()
#     alices_loss.backward()
#     alices_opt.step()

    
#     print(f"Alice loss {alices_loss.get().data}")
#     print(f"Bob loss {bobs_loss.get().data}")
#     print('-'*20)

In [0]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [0]:
with th.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data)/2).get())

# Additive sharing 

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

compute_nodes = (bob, alice)

In [0]:
bob.clear_objects()
alice.clear_objects()
crypto_provider.clear_objects()

<VirtualWorker id:crypto_provider #objects:0>

In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,10)
        
    def forward(self,x):
        x = x.view(x.shape[0],-1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x),dim=1)
    
        return x
        

In [0]:
def create_federated_data_loaders(workers):

    # Define a transform to normalize the data
    transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))])
    # Download and load the training data
    mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

    trainloader = th.utils.data.DataLoader(mnist_trainset, batch_size=32)
    remote_dataset = (list(),list())
    for i, (data, target) in enumerate(trainloader):
        worker_index = i % len(workers)
        data = data.send(workers[worker_index])
        target = target.send(workers[worker_index])
        remote_dataset[worker_index].append((data,target))

    # Download and load the test data
    mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    # Set up the testloader
    
    testloader = th.utils.data.DataLoader(mnist_testset, batch_size=1000, shuffle=False)


    return remote_dataset, testloader

In [0]:
remote_dataset , testloader = create_federated_data_loaders(compute_nodes)

0it [00:00, ?it/s]

9920512it [00:02, 3399630.63it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 48817.32it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:02, 816402.12it/s]                             
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 18510.46it/s]            


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
def update(data, target, model, opt, criterion):
    model.send(data.location)
    opt.zero_grad()
    pred = model(data)
    loss = criterion(pred, target)
    loss.backward()
    opt.step()
    return model, loss

In [0]:
bobs_model = Net()
alices_model = Net()

lr = 1e-2
bobs_opt = optim.SGD(bobs_model.parameters(), lr=lr)
alices_opt = optim.SGD(alices_model.parameters(), lr=lr)


In [0]:
models = [bobs_model, alices_model]
opts = [bobs_opt, alices_opt]
params = [list(bobs_model.parameters()), 
          list(alices_model.parameters())]

In [0]:
def train(epoch):
    for e in range(epoch):
        print(f"{e}/{epoch}")
        running_loss = 0
        for data_i in range(len(remote_dataset[0])-1):
            # train a little bit
            for remote_i in range(len(compute_nodes)):
                data, target = remote_dataset[remote_i][data_i]
                models[remote_i], loss = update(data, target, models[remote_i], 
                                          opts[remote_i], nn.NLLLoss())
                running_loss += loss.get()


            # aggregate params using additive sharing
            new_params = []
            for param_i in range(len(params[0])):
                spdz_params = list()
                for remote_i in range(len(compute_nodes)):
                    # + 0 means copy
                    remote_param = (params[remote_i][param_i] + 0).fix_precision()
                    shared_param = remote_param.share(bob, alice, crypto_provider=crypto_provider).get()
                    spdz_params.append(shared_param)
                    
                # average the params
                new_param = (spdz_params[0] + spdz_params[1]).get().float_precision()/2
                new_params.append(new_param)
            
            
            # set the param to new trained params
            with th.no_grad():
                for model in params:
                    for param in model:
                        param *= 0

                for model in models:
                    model.get()

                for remote_i in range(len(compute_nodes)):
                    # set the new params for all worker's model
                    for param_index in range(len(params[remote_i])):
                        params[remote_i][param_index].set_(new_params[param_index])
        print(f"Loss {running_loss/len(remote_dataset[0])}")

In [0]:
def test():
    model = models[0].eval().get()
    preds = th.tensor([])
    for data, label in testloader:
        out = model(data)
        pred = th.argmax(th.exp(out),dim=1) == label
        preds = th.cat([preds,pred.float()])
    print(f"Acc : {preds.float().mean()*100}")

In [0]:
train(5)

0/5


In [0]:
test()